# `Nuqleon.Linq.Expressions.Bonsai.Serialization`

Provides serialization of Bonsai expressions in JSON and binary form.

## Reference the library

### Option 1 - Use a local build

If you have built the library locally, run the following cell to load the latest build.

In [ ]:
#r "bin/Debug/net6.0/Nuqleon.Linq.Expressions.Bonsai.Serialization.dll"

### Option 2 - Use NuGet packages

If you want to use the latest published package from NuGet, run the following cell.

In [ ]:
#r "nuget:Nuqleon.Linq.Expressions.Bonsai.Serialization,*-*"

## (Optional) Attach a debugger

If you'd like to step through the source code of the library while running samples, run the following cell, and follow instructions to start a debugger (e.g. Visual Studio). Navigate to the source code of the library to set breakpoints.

In [ ]:
System.Diagnostics.Debugger.Launch();

## Expression Serialization using Bonsai

Expression trees can be serialized to the JSON-based Bonsai format using the `BonsaiExpressionSerializer` class. First, we'll create a simple expression tree and convert it to the `ExpressionSlim` representation.

In [ ]:
using System.Linq.Expressions;

Expression<Func<string, int>> expr = s => s.Length;

var slim = expr.ToExpressionSlim();

Console.WriteLine(slim);

Lambda(MemberAccess(System.String.Length, Parameter(System.String, s)), Parameter(System.String, s))


To serialize this expression tree, we can use the simplest overload of the `BonsaiExpressionSerializer`, as shown below. By calling the `Serialize` method, we obtain a `string` holding the Bonsai JSON-based format.

In [ ]:
using System.Linq.Expressions.Bonsai.Serialization;

var ser = new BonsaiExpressionSerializer();

string json = ser.Serialize(slim);

Console.WriteLine(json);

{"Context":{"Members":[["P",0,"Length",[],2]],"Types":[["::","System.String",0],["::","System.Func`2",0],["::","System.Int32",0],["<>",1,[0,2]]],"Assemblies":["System.Private.CoreLib, Version=5.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e"],"Version":"0.9.0.0"},"Expression":["=>",3,[".",0,["$",0,0]],[[0,"s"]]]}


In order to inspect the Bonsai tree in some more detail, we'll use the `PrettyPrinter` type from `Nuqleon.Json`, and print the tree again.

In [ ]:
using Json = Nuqleon.Json.Expressions;

var print = new Json.PrettyPrinter();

string pretty = print.Visit(Json.Expression.Parse(json));

Console.WriteLine(pretty);

{
  "Context": {
    "Members": [
      [
        "P",
        0,
        "Length",
        [
          
        ],
        2
      ]
    ],
    "Types": [
      [
        "::",
        "System.String",
        0
      ],
      [
        "::",
        "System.Func`2",
        0
      ],
      [
        "::",
        "System.Int32",
        0
      ],
      [
        "<>",
        1,
        [
          0,
          2
        ]
      ]
    ],
    "Assemblies": [
      "System.Private.CoreLib, Version=5.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e"
    ],
    "Version": "0.9.0.0"
  },
  "Expression": [
    "=>",
    3,
    [
      ".",
      0,
      [
        "$",
        0,
        0
      ]
    ],
    [
      [
        0,
        "s"
      ]
    ]
  ]
}


From a top-level point of view, a Bonsai tree consists of two sections:

* `Context` which holds information about assemblies, types, and members.
* `Expression` which resembles a LISP S-expression where every node of an expression tree is represented as a JSON array whose first element is a discriminator.

Taking a closer look, our expression looks like:

```json
[
    "=>",
    3,
    [
        ".",
        0,
        ["$", 0, 0]
    ],
    [
        [0, "s"]
    ]
]
```

The top-level node is of type `=>` which represents a lambda and consists of three elements:

* `3` represents the type of the lambda, which corresponds to index `3` in the `Types` table in the context, which we'll look at in a moment;
* the body of the lambda;
* an array of parameters to the lambda.

Starting from the parameters at the end, we have an array with a single element `[0, "s"]`. In here, `s` is the name of the parameter, and `0` is the type, which is again an entry in the `Types` tables in the context. Looking at the `Types` table, we can see four entries:

```json
    ["::", "System.String", 0],
    ["::", "System.Func`2", 0],
    ["::", "System.Int32", 0],
    ["<>", 1, [0, 2]]
```

The first three entries are all of type `::` which denotes a *simple type*, which is just a type with a name located in an assembly with the specified index, in all cases above `0`, which corresponds to the `Assemblies` table's first entry:

```
    "System.Private.CoreLib, Version=5.0.0.0, Culture=neutral, PublicKeyToken=7cec85d7bea7798e"
```

> **Note**: Assemblies are just a .NET artifact but Bonsai trees can also run on different platforms, e.g. be bound to a C++ runtime, get evaluated in JavaScript, etc. For interoperability purposes, one often erases assemblies for well-known identifiers that are agreed upon by all target systems (e.g. `STD` to represent a standard library).

Back to our type table, we have three simple types, for `String`, `Func<,>`, and `Int32`. The fourth type is of type `<>` and reperesents a closed generic type:

```json
    ["<>", 1, [0, 2]]
```

This notation means that we're closing over the open generic type at index `1` in the table (`Func<,>`) using type arguments at index `0` (`String`) and index `2` (`Int32`). As a result, this fourth entry in the type table (i.e. at index `3`) represents `Func<String, Int32>`, which is indeed the type of the lambda. Armed with this knowledge, we can also decipher the parameter `[0, "s"]` whose type is at index `0` (`String`).

```json
[
    "=>",
    3, // Func<string, int>
    [
        ".",
        0,
        ["$", 0, 0]
    ],
    [
        [0 /* string */, "s"]
    ]
]
```

Finally, we can have a look at the body of the lambda:

```json
    [
        ".",
        0,
        ["$", 0, 0]
    ]
```

This node's discriminator is `.` which denotes member lookup. The next array slot contains the member from the `Members` table, which contains:

```json
    ["P", 0, "Length", [], 2]
```

In here, `P` stands for property, which consists of a declaring type (`0` in the `Types` table, i.e. `String`), a name (`Length`), indexer parameters (none), and the return type (`2` in the `Types` table, i.e. `Int32`).

Finally, the last array slow in the member lookup node is `["$", 0, 0]`, which is a variable reference denoted by `$`. The indexes `0` and `0` refer to the scope and the index of the variable in that scope. In this case, the immediately surrounding scope is the lambda, whose first parameter is `string s`, so this expression node refers to that variable.

## Expression Deserialization using Bonsai

Expression trees can be deserialized from the JSON-based Bonsai format using the `BonsaiExpressionSerializer` class. Building off one of the earlier examples, we'll create a simple expression tree, convert it to the `ExpressionSlim` representation and serialize it to a string. Then we'll look at how to deserialize it from that string to expression, compile that expression to a delegate we can pass arguments into and then compute a result using our deserialized expression.

In [2]:
using System.Linq.Expressions.Bonsai.Serialization;

var ser = new BonsaiExpressionSerializer();

//Serialize the expression below
Expression<Func<List<int>, int>> expr = x => x.Max();
var slim = expr.ToExpressionSlim();
var json = ser.Serialize(slim);

//Deserialize and calculate a result
var deserializedSlim = ser.Deserialize(json);
var deserializedExpression = (Expression<Func<List<int>, int>>)deseralizedSlim.ToExpression();
var exprDelegate = deserializedExpression.Compile();
var data = new List<int>{1, 2, 3, 4, 5};
var result = exprDelegate(data); // 5